# How to Convert Keras to ONNX

The following tutorial is a brief example of how to convert a [Keras](https://keras.io/) or Tensor ML model to [ONNX](https://onnx.ai/ ).  This allows organizations that have trained Keras or Tensor models to convert them and use them with Wallaroo.

This tutorial assumes that you have a Wallaroo instance and are running this Notebook from the Wallaroo Jupyter Hub service.  This sample code is made to work with the Wallaroo IMDB demo.  The models converted from this procedure can directly be used in that demonstration.

* **IMPORTANT NOTE**:  As of this time, this tutorial does not work in a Wallaroo instance.  The instructions have been geared towards converting these models in a separate Python environment.  Note that there may be warnings displayed based on whether you have a GPU installed in the target machine - these can be ignored.

This tutorial provides the following:

* `embedder`: akes pre-tokenized text documents (model input: 100 integers/datum; output 800 numbers/datum) and creates an embedding from them.  It has the input signature name of `input`.
* `sentiment_model`: The second model classifies the resulting embeddings from 0 to 1, which 0 being an unfavorable review, 1 being a favorable review.  This has 800 inputs.  It has the input signature name of `embeddings`.

The following libraries need to be installed in your Python environment:
    
* onnxconverter-common
* numpy
* keras
* tensorflow
* onnxruntime
* tf2onnx
* onnx

These can be installed through either `pip` or `conda`, depending on your Python environment.  For example:

```
pip install numpy keras tensorflow onnxruntime tf2onnx onnx onnxconverter-common
```

Once installed, they can be imported into your Python code:

In [1]:
import numpy
import keras
from tensorflow.keras import *
from tensorflow.keras.layers import *

import tensorflow as tf
import onnxruntime
import tf2onnx
import onnx

2022-04-22 18:53:25.410680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 18:53:25.410716: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load the Models

The first step is to load the models via the `keras.models.load_model` method:

In [2]:
embedder = keras.models.load_model('embedder')
sentiment = keras.models.load_model('sentiment_model')

2022-04-22 18:53:26.989602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-22 18:53:26.989644: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-22 18:53:26.989666: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wallaroo): /proc/driver/nvidia/version does not exist
2022-04-22 18:53:26.989861: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Retrieve the Opset

The following code will retrieve the correct Onnx Opset Version that will be used for the conversion.

In [3]:
# figure out the correct opset

from onnx.defs import onnx_opset_version
from onnxconverter_common.onnx_ex import DEFAULT_OPSET_NUMBER
TARGET_OPSET = min(DEFAULT_OPSET_NUMBER, onnx_opset_version())
TARGET_OPSET

15

## Convert the Model

The model conversion uses the method `tf2onnx.convert.from_keras(model, insert_signature, opset)` which takes the following arguments as detailed in the [tensorflow-onnx](https://github.com/onnx/tensorflow-onnx):

* `model`: the tf.keras model we want to convert
* `input_signature`: a tf.TensorSpec or a numpy array defining the shape/dtype of the input
* `opset`: the opset to be used for the ONNX model, default is the latest

In the following example, the `input_signature` is set first for the `embedder` conversion, then for the `sentiment_model` conversion.


In [4]:
input_signature = [tf.TensorSpec([None, 100], tf.float32, name='input')]

onnx_model_embedded, _  = tf2onnx.convert.from_keras(embedder, input_signature, opset=TARGET_OPSET)


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-04-22 18:53:27.475655: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-22 18:53:27.475803: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-04-22 18:53:27.514245: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-04-22 18:53:27.533604: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-22 18:53:27.533714: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-04-22 18:53:27.536251: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 11 nodes (0), 11 edges (0

In [5]:
input_signature = [tf.TensorSpec([None, 800], tf.float32, name='embeddings')]
onnx_model_sentiment, _  = tf2onnx.convert.from_keras(sentiment, input_signature, opset=TARGET_OPSET)


2022-04-22 18:53:27.594728: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-22 18:53:27.594816: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-04-22 18:53:27.595529: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.006ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.

2022-04-22 18:53:27.609848: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-22 18:53:27.609944: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-04-22 18:53:27.611573: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 8 nodes (-2), 9 edges (-2

## Save the Models

Once converted, the models can be saved with the `onnx.save(model, filename)` method with takes the following arguments:

* model: The model to save.
* filename:  The file name to save the model to.

In [6]:
onnx.save(onnx_model_embedded, 'embedder.onnx')

onnx.save(onnx_model_sentiment, 'sentiment_model.onnx')